In [ ]:
! pip install azure-search-documents --pre
! pip install python-dotenv

In [1]:
# Import required libraries  
import os  
import json  
import openai
import keys
from dotenv import load_dotenv  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,  
)  
  
# Configure environment variables  
load_dotenv()  
service_endpoint = "https://rsaf-cognitive-search-service.search.windows.net"
index_name = "rsaf-cognitive-search"
key = keys.cognitive_key
openai.api_type = "azure"  
openai.api_key = "0e576c031e5446e19cc6d866ad3d9f3f"
openai.api_base = "https://raid-ses-openai.openai.azure.com/"
openai.api_version = "2023-05-15"
credential = AzureKeyCredential(key)

In [2]:
client_index = SearchIndexClient(endpoint=service_endpoint, credential=credential)

In [37]:
for i in client_index.list_indexes():
    print(i.name)

llamaindex-demo


In [80]:
search_client = SearchClient(endpoint=service_endpoint, index_name="llamaindex-demo", credential=credential)

In [87]:
results = search_client.search(search_text="Advanced Transitions confined spaces", top=3)

In [88]:
for i in results:
    print(i)

{'content': 'OFFICIAL (CLOSED)   \n13-1 \nOFFICIAL (CLOSED)   PAR T B \n \nCHAPTER 13  \n \nADVANCED TRANSITIONS  \n \nINTRODUCTION  \n \n13.1 Helicopter operations involve landing and taking off from confined \nspaces, high altitudes or at a high AUW. In all of these conditions, it may not be \nfeasible to employ the basic transition technique due  to limitations in the engine \npower available or availability of forward distance. In such scenarios, advanced \ntransition techniques are employed.  \n \n13.2 Objective . The objectives are; (1) determine the power margin available \nthrough hover and forward flight po wer checks, (2) select the appropriate \nadvanced transition technique, (3) conduct a OGE, IGE, cushion creep and \nrunning take off transition to the climb and (4) conduct a transition from forward \nflight to an OGE hover, zero speed and running landing.  \n \n \nPOWER DEMANDS IN A HELICOPTER  \n \n13.3 Prior to  learning advanced transition techniques, an understand ing 

In [72]:
retrieve_ids[0]['@search.score']

26.067549

In [53]:
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="swiftfaq-ada002")
    embeddings = response['data'][0]['embedding']
    return embeddings


In [55]:
# Semantic Hybrid Search
query = "what is autorotation?"

index_name = "llamaindex-demo"

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")  

results = search_client.search(  
    search_text=query,  
    vectors=[vector],
    query_type="semantic", query_language="en-us", semantic_configuration_name='my-semantic-config', query_caption="extractive", query_answer="extractive",
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

HttpResponseError: (FeatureNotSupportedInService) Semantic search is not enabled for this service.
Parameter name: queryType
Code: FeatureNotSupportedInService
Message: Semantic search is not enabled for this service.
Parameter name: queryType
Exception Details:	(SemanticQueriesNotAvailable) Semantic search is not enabled for this service.
	Code: SemanticQueriesNotAvailable
	Message: Semantic search is not enabled for this service.